In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os
import pandas as pd

# Data Diri
print("-------------------------------------------------------------------------------------------\n")
nama = "Ananda Rauf Maududi"
school_title = "Strata 1 / Sarjana Komputer"
school = "Universitas Bina Sarana Informatika"
job_title = "Junior Python Programmer"
work_on = "PT Kalanara Group Indonesia"
print(nama)
print(school_title)
print(school)
print(job_title)
print(work_on)
print("-------------------------------------------------------------------------------------------\n")

# Direktori dataset
base_dir = '/content/dataset'
train_dir = os.path.join(base_dir, 'train_validation')
validation_dir = os.path.join(base_dir, 'train_validation')

# Augmentasi Gambar menggunakan ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

# Membangun Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Kompilasi Model
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-4),
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5)

# Pelatihan Model
history = model.fit(
    train_generator,
    steps_per_epoch=100,  # asumsikan kita memiliki 2000 gambar, batch_size=20
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50,  # asumsikan kita memiliki 1000 gambar validasi, batch_size=20
    callbacks=[early_stopping, reduce_lr])

# Menyimpan Riwayat Pelatihan
history_df = pd.DataFrame(history.history)
history_df.to_csv('/content/Save_Dataset/training_history.csv', index=False)

# Evaluasi Model
eval_result = model.evaluate(validation_generator)
print(f'Validation Loss: {eval_result[0]}, Validation Accuracy: {eval_result[1]*100:.2f}%')

# Fungsi Prediksi Gambar yang Diunggah
def predict_uploaded_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    predicted_class = 'Kertas' if predictions[0] > 0.5 else 'Bukan Kertas'
    return predicted_class

# Prediksi Gambar
uploaded_image_path = '/content/gambar_upload/Kertas.png'
prediction = predict_uploaded_image(uploaded_image_path)
print(f'Prediction result: {prediction}')

-------------------------------------------------------------------------------------------

Ananda Rauf Maududi
Strata 1 / Sarjana Komputer
Universitas Bina Sarana Informatika
Junior Python Programmer
PT Kalanara Group Indonesia
-------------------------------------------------------------------------------------------

Found 401 images belonging to 3 classes.
Found 401 images belonging to 3 classes.


Epoch 1/100
 13/100 [==>...........................] - ETA: 17s - loss: -248.1658 - accuracy: 0.0524

13/13 [==============================] - 1s 85ms/step - loss: -1332.2748 - accuracy: 0.0474
Validation Loss: -1332.2747802734375, Validation Accuracy: 4.74%
1/1 [==============================] - 0s 73ms/step
Prediction result: Kertas
